# Analysing Amazon Reviews with Pandas II: Finding controversial reviews
(2017-02-03 (C) Wouter van Atteveldt CC-BY-SA)

With the amazon reviews data prepared in Part I, we can start doing substantive queries.

There are many interesting questions that can be answered with this data.
Similar to the previous handout, the one we'll answer here is: what are the most controversial reviews?

# Review statistics

Before calculating the 'controversiality', let's first look at some simple statistics. 
Let's count the number of reviews per product, which is automatically sorted in descending order:

In [1]:
import pandas as pd
reviews = pd.read_pickle("reviews.pickle")
counts = reviews.asin.value_counts()
counts.head()

B000FEH8ME    742
B002IEVJRY    741
B00934WBRO    476
B006MONQMC    468
B003GTR8IO    445
Name: asin, dtype: int64

And let's get the mean review and counts per score:

In [2]:
reviews.overall[reviews.asin == counts.index[0]].mean()

3.8544474393530996

In [3]:
reviews.overall[reviews.asin == counts.index[0]].value_counts()

4.0    261
5.0    243
3.0    151
2.0     61
1.0     26
Name: overall, dtype: int64

Now, we can compute the mean score per product for all products using the `groupby` function:

In [4]:
stats = reviews.groupby(['asin'])['overall'].agg(['count', 'mean'])
stats.head()

,count,mean
asin,,
616719923X,16,4.062500
9742356831,15,4.733333
B00004S1C5,7,4.142857
B0000531B7,9,4.555556
B00005344V,7,4.428571


# Controversiality: Using a custom aggregation function

The above code computed the mean score per product. 
We can also supply a custom function to the aggregate, which takes the values as its argument.
Similar to the previous handout,
we compute the controversiality by taking the lower count of the number of good (5-star) and bad (1 or 2 star) reviews:

In [5]:
def controversiality(x):
    c = x.value_counts()
    return min(c.get(5, 0), c.get(1, 0) + c.get(2, 0))

Let's test this with the most reviewed product identified above, which had 261 good and 61+26=87 bad reviews:

In [6]:
controversiality(reviews.overall[reviews.asin == 'B000FEH8ME'])

87

Now, we can apply this to all reviews and see the most controversial ones:

In [7]:
c = reviews.groupby(['asin'])['overall'].agg([controversiality])
c = c.sort_values("controversiality", ascending=False)
c.head()

,controversiality
asin,
B002IEVJRY,90.0
B000FEH8ME,87.0
B004YV80O4,72.0
B007B9WAH4,70.0
B006MONQMC,68.0


# Controversiality: using vectorized functions

The above code works, but it runs pretty slow, more than 5 seconds on my computer.
The reason for this is that for each of the 9000 reviews, pandas first makes a list of all scores, 
and then calls our python function with that list.
Effectively, we're placing our python code within the implicit for-loop. 
Since python code is easily 10-100 times slower than the compiled C or Fortran code used internally in numpy, 
this is always much slower than using operations that work on the whole set of reviews at once.
Such operations are ofen called 'vectorized' functions (since they are called on the whole vector, rather than on each element of the vector)

To do this for controversiality, we use a different strategy:
We first compute the number of good and bad reviews per products by filtering on score and using value_counts:

In [8]:
good = reviews.asin[reviews.overall == 5].value_counts()
bad = reviews.asin[reviews.overall <= 2].value_counts()
good.head()

B000E1D7RS    260
B000FEH8ME    243
B00DS842HS    220
B0051COPFI    203
B0051SU0OW    202
Name: asin, dtype: int64

Next, we use the `combine` function. 
This function allows us to call a function (in this case: min) on the combination of the values from two series.
Since not every product will have bad (or good) reviews, we specify a fill_value of 0 to prevent NaN's when a value is missing. 

In [9]:
c = good.combine(bad, func=min, fill_value=0)
c = c.sort_values(ascending=False)
c.head()

B002IEVJRY    90
B000FEH8ME    87
B004YV80O4    72
B007B9WAH4    70
B006MONQMC    68
Name: asin, dtype: int64

This is much better, but you can still notice that the combine function takes time. 
The reason for this is that 'min' is a python function, which is still called on each of the rows.
So, while the largest time consumer in the first attempt (creating the lists of scores per product and counting them) is  removed, we are still running a python function on each row.

To get rid of this, we can also concatenate the two series into a dataframe and use the df.min function to compute the row minima.
Note that we specify axis=1 on both the concat and min to state that we wish to concatenate columns (rather than rows) and get row totals (rather than column totals)

In [10]:
combined = pd.concat({"good":good, "bad":bad}, axis=1)
c = combined.min(axis=1)
c.head()

616719923X     1.0
9742356831    12.0
B00004S1C5     1.0
B0000531B7     6.0
B00005344V     5.0
dtype: float64

# Combining data frames

We've computed a number of useful statistics about the products, such as average review and controversiality.
In many cases, it is useful to create a combined data frame of the product details and statistics,
so we can e.g. easily see the title of succesful or controversial products.

There are a number of ways to do this.
Since all statistics have the asin as the index, which is also the index of the products data frame, we can simply assign e.g. the controversiality to a new column:

In [11]:
products = pd.read_pickle("products.pickle")
products['controversiality'] = c
products = products.sort_values("controversiality", ascending=False)
products.head()

,brand,categories,description,imUrl,price,related,salesRank,title,controversiality
asin,,,,,,,,,
B002IEVJRY,illy issimo,[[Grocery & Gourmet Food]],,http://ecx.images-amazon.com/images/I/419ezkZZ...,24.77,"{'bought_together': ['B00IHVHM4Q', 'B000LL0R8I...",{'Grocery & Gourmet Food': 3632},"illy issimo Coffee Drink, Cappuccino, 8.45-Oun...",90.0
B000FEH8ME,Pure Bar,[[Grocery & Gourmet Food]],Our Chocolate Brownie Bar is made with fair tr...,http://ecx.images-amazon.com/images/I/41bFHm7a...,23.50,"{'bought_together': ['B008JKU2CO', 'B008JKUBZM...",{'Grocery & Gourmet Food': 1523},"Pure Bar Organic Chocolate Brownie, Gluten Fre...",87.0
B004DIR3TQ,Scharffen Berger,[[Grocery & Gourmet Food]],,http://ecx.images-amazon.com/images/I/41kvVyGr...,20.50,"{'bought_together': ['B004NSKWMQ', 'B00D8TSUAM...",{'Grocery & Gourmet Food': 37338},"Scharffen Berger Chocolate Bar, Bittersweet Da...",80.0
B003OGKCDC,NaN,"[[Grocery & Gourmet Food, Cooking & Baking, Co...",NaN,NaN,NaN,NaN,NaN,NaN,74.0
B004YV80O4,Velveeta,[[Grocery & Gourmet Food]],Velveeta Ultimate Cheeseburger Mac Dinner Kit....,http://ecx.images-amazon.com/images/I/51WN9-R4...,29.97,"{'bought_together': ['B0095F9EK8', 'B0019RLJV2...",{'Grocery & Gourmet Food': 15130},Kraft Velveeta Ultimate Cheesburger Skillets D...,72.0


We can also directly join a dataframe such as the 'stats' created above to the products:

In [12]:
products = products.join(stats)
products.head()

,brand,categories,description,imUrl,price,related,salesRank,title,controversiality,count,mean
asin,,,,,,,,,,,
B002IEVJRY,illy issimo,[[Grocery & Gourmet Food]],,http://ecx.images-amazon.com/images/I/419ezkZZ...,24.77,"{'bought_together': ['B00IHVHM4Q', 'B000LL0R8I...",{'Grocery & Gourmet Food': 3632},"illy issimo Coffee Drink, Cappuccino, 8.45-Oun...",90.0,741,3.731444
B000FEH8ME,Pure Bar,[[Grocery & Gourmet Food]],Our Chocolate Brownie Bar is made with fair tr...,http://ecx.images-amazon.com/images/I/41bFHm7a...,23.50,"{'bought_together': ['B008JKU2CO', 'B008JKUBZM...",{'Grocery & Gourmet Food': 1523},"Pure Bar Organic Chocolate Brownie, Gluten Fre...",87.0,742,3.854447
B004DIR3TQ,Scharffen Berger,[[Grocery & Gourmet Food]],,http://ecx.images-amazon.com/images/I/41kvVyGr...,20.50,"{'bought_together': ['B004NSKWMQ', 'B00D8TSUAM...",{'Grocery & Gourmet Food': 37338},"Scharffen Berger Chocolate Bar, Bittersweet Da...",80.0,123,4.544715
B003OGKCDC,NaN,"[[Grocery & Gourmet Food, Cooking & Baking, Co...",NaN,NaN,NaN,NaN,NaN,NaN,74.0,83,4.855422
B004YV80O4,Velveeta,[[Grocery & Gourmet Food]],Velveeta Ultimate Cheeseburger Mac Dinner Kit....,http://ecx.images-amazon.com/images/I/51WN9-R4...,29.97,"{'bought_together': ['B0095F9EK8', 'B0019RLJV2...",{'Grocery & Gourmet Food': 15130},Kraft Velveeta Ultimate Cheesburger Skillets D...,72.0,428,3.616822


To get slightly nicer output, we only want to see the title and our three new columns.
This is achieved by using products.columns to get a list of column names, and then selecting the last three and adding them to a list containing only the 'title' name:

In [13]:
products[["title"] + list(products.columns[-3:])].head()

,title,controversiality,count,mean
asin,,,,
B002IEVJRY,"illy issimo Coffee Drink, Cappuccino, 8.45-Oun...",90.0,741,3.731444
B000FEH8ME,"Pure Bar Organic Chocolate Brownie, Gluten Fre...",87.0,742,3.854447
B004DIR3TQ,"Scharffen Berger Chocolate Bar, Bittersweet Da...",80.0,123,4.544715
B003OGKCDC,NaN,74.0,83,4.855422
B004YV80O4,Kraft Velveeta Ultimate Cheesburger Skillets D...,72.0,428,3.616822


Of course, in this case it is actually much easier since the title is actually the last column, so we can simply select the last four columns with the .ix function, using the unspecified `:` to select all rows:

In [14]:
products.ix[:, -4:].head()

,title,controversiality,count,mean
asin,,,,
B002IEVJRY,"illy issimo Coffee Drink, Cappuccino, 8.45-Oun...",90.0,741,3.731444
B000FEH8ME,"Pure Bar Organic Chocolate Brownie, Gluten Fre...",87.0,742,3.854447
B004DIR3TQ,"Scharffen Berger Chocolate Bar, Bittersweet Da...",80.0,123,4.544715
B003OGKCDC,NaN,74.0,83,4.855422
B004YV80O4,Kraft Velveeta Ultimate Cheesburger Skillets D...,72.0,428,3.616822


# Comparing performance 

The final solution for computing controversiality seems a lot faster,
but it would be nice to measure how much faster it is. 
To do this, we can use the `%timeit` 'magic' function built into ipython. 
Timeit runs an expression *n* times to measure how long it takes. 
It then does this 3 times and reports the best of these 3 (to prevent e.g. a background process from interfering too much). 

In [15]:
%timeit -n 1 reviews.groupby(['asin'])['overall'].agg([controversiality])

1 loop, best of 3: 4.59 s per loop


In [16]:
%timeit -n 10 good.combine(bad, func=min, fill_value=0)

10 loops, best of 3: 297 ms per loop


In [17]:
%timeit -n 10 pd.concat({"good":good, "bad":bad}, axis=1).min(axis=1)

10 loops, best of 3: 7.58 ms per loop


So, the first change (using combine and min instead of a custom aggregation function) gave a speedup of around 16x. 
The second improvement (using vectorized df.min instead of implicitly looping the python min function) also gives a very significant speedup of around 36x. 
In total, the difference between the first attempt and the final solution is over 500x speedup.

This might not be significant on toy data sets, but if you go into the millions of rows it can be very important. 
So, always try to avoid calling python functions on individual rows but try to use vectorized functions like df.min.
And know that you can always used `%timeit` to profile your code.
It is best to only improve performance where it actually matters, as the fastest code is not always the most readable code, and this matters if you need to reuse or adapt the code years later.